In [2]:
import os, requests, time

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from urllib.parse import urlparse, urljoin

In [4]:
nl_files_list = os.listdir("../met-naam-2018/castles-netherlands/")

In [5]:
nl_urls = ['https://www.'+item[:-6].lstrip('www')+'.'+item.rstrip('.txt')[-2:] for item in nl_files_list]

In [6]:
nl_urls[0:10]

['https://www.buitenplaatsbeeckestijn.nl',
 'https://www.muiderslot.nl',
 'https://www.dekemastate.nl',
 'https://www.menkemaborg.nl',
 'https://www.sypesteyn.nl',
 'https://www.ruinevanbrederode.nl',
 'https://www.fraeylemaborg.nl',
 'https://www.middachten.nl',
 'https://www.kasteelamerongen.nl',
 'https://www.twickel.nl']

In [6]:
fr_files_list = os.listdir("../met-naam/castles-france/")

In [20]:
domain_extensions = ['com', 'net', 'org', 'bzh']
# what about .corsica???
fr_urls_div = [link[:-4] for link in fr_files_list if link.endswith('.txt') and link[-7:-4] in domain_extensions]
fr_urls_div = ['https://www.'+link[:-3].lstrip('www') + '.' + link[-3:] for link in fr_urls_div]

fr_urls_fr = [link[:-4] for link in fr_files_list if link.endswith('.txt') and link[-6:-4]== 'fr']
fr_urls_fr = ['https://www.'+link[:-3].lstrip('www') + '.' + link[-3:] for link in fr_urls_fr]

#catch the unusual .corsica domains
fr_urls_corsica = [link[:-4] for link in fr_files_list if link.endswith('.txt') and link[-11:-4]== 'corsica']
fr_urls_corsica = ['https://'+link[:-7].lstrip('www') + '.' + link[-7:] for link in fr_urls_corsica]

fr_urls = fr_urls_div + fr_urls_fr + fr_urls_corsica

print(len(fr_files_list), fr_files_list[:20])
print(len(fr_urls), fr_urls[:20])

92 ['chateaudedigoinefr.txt', 'wwwchateauangersfr.txt', 'chateaudutaureaubzh.txt', 'wwwchateaudejosselincom.txt', 'wwwallymesnet.txt', 'careilcom.txt', 'wwwchateau-ancycom.txt', 'susciniofr.txt', 'wwwchateaudekerioletcom.txt', 'wwwchateauvillandryfr.txt', 'wwwchateaubrissacfr.txt', 'wwwchateauguillaumeleconquerantfr.txt', 'wwwcitadelledesisteronfr.txt', '.DS_Store', 'wwwchateau-de-st-fargeaucom.txt', 'wwwchateauramburespicardiefr.txt', 'wwwchateaubaugefr.txt', 'wwwchateaudebloisfr.txt', 'wwwchateaudelarocheguyonfr.txt', 'wwwchateau-lafayettecom.txt']
91 ['https://www.chateaudutaureau.bzh', 'https://www.chateaudejosselin.com', 'https://www.allymes.net', 'https://www.careil.com', 'https://www.chateau-ancy.com', 'https://www.chateaudekeriolet.com', 'https://www.chateau-de-st-fargeau.com', 'https://www.chateau-lafayette.com', 'https://www.chateaudecoupiac.com', 'https://www.chateau-savigny.com', 'https://www.chateaugratot.com', 'https://www.domainedechantilly.com', 'https://www.chateau-bau

### Scrape function

In [ ]:
def scrape_website(url, visited_urls=set()):
    try:
        if url in visited_urls:
            return ""
        
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            visited_urls.add(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            content = soup.get_text()
            
            # Find all internal links and recursively scrape them
            internal_links = soup.find_all('a', href=True)
            for link in internal_links:
                absolute_url = urljoin(url, link['href'])
                if urlparse(absolute_url).netloc == urlparse(url).netloc:
                    content += scrape_website(absolute_url, visited_urls)
            
            return content
        elif response.status_code == 403:
            print(f"403 Forbidden: {url}")
            return ""
        else:
            print(f"Error scraping {url}: {response.status_code}")
            return ""
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return ""

def write_to_file(content, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)

Scraped https://www.buitenplaatsbeeckestijn.nl and wrote content to ../met-naam-2024/castles-netherlands/www.buitenplaatsbeeckestijn.nl.txt
Error scraping https://www.muiderslot.nl/collectie/digitale-boekenkast/: 404
Error scraping https://www.muiderslot.nl/collectie/digitale-boekenkast/: 404
Error scraping https://www.muiderslot.nl/collectie/schatten/: 404
Error scraping https://www.muiderslot.nl/collectie/schatten/: 404
Error scraping https://www.muiderslot.nl/collectie/schatten/: 404
Error scraping https://www.muiderslot.nl/collectie/schatten/: 404
Error scraping https://www.muiderslot.nl/collectie/schatten/: 404
Error scraping https://www.muiderslot.nl/collectie/schatten/: 404
Error scraping https://www.muiderslot.nl/collectie/schatten/: 404
Error scraping https://www.muiderslot.nl/collectie/schatten/: 404
Error scraping https://www.muiderslot.nl/collectie/schatten/: 404
Error scraping https://www.muiderslot.nl/collectie/schatten/: 404
Error scraping https://www.muiderslot.nl/colle

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [ ]:
# Get all Dutch websites or a range of urls list
write_path="../met-naam-2024/castles-netherlands/"

def main():
    for url in nl_urls[0:6]:
        try:
            domain = urlparse(url).netloc
            filename = write_path+f"{domain}.txt"
            content = scrape_website(url)
            if content:
                write_to_file(content, filename)
                print(f"Scraped {url} and wrote content to {filename}")
            else:
                print(f"Failed to scrape {url}")
            # Introduce a delay between requests to avoid overwhelming the server
            time.sleep(2)
        except Exception as e:
            print(f"Error processing {url}: {e}")

if __name__ == "__main__":
    main()

#### Test one url in Wayback machine

In [104]:
url= "https://web.archive.org/web/20180511012246/https://www.muiderslot.nl/"

In [106]:
response1 = requests.get(url)

In [107]:
soup1 = BeautifulSoup(response.text)

In [108]:
allowed_tags = ['div','p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'span', 'br']
all_text = ' '.join([element.get_text() for element in soup.find_all(allowed_tags)])
with open(url[29:33], "w") as file:
    file.write(all_text)

## Check up functions 

In [112]:
def check_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print(0)
        else:
            print(f"The URL '{url}' is unreachable. Status code: {response.status_code}")
    except requests.ConnectionError:
        print(f"The URL '{url}' is unreachable.")

for i in range(len(fr_urls)):
    check_url(fr_urls[i])


0
0
0
0
0
0
0
0
0
0
0
0
0
The URL 'https://www.chateau-eaucourt.com' is unreachable. Status code: 429
0
0
0
0
0
0
0
0
0
0
0
0
0
The URL 'https://www.chateaudauphin.com' is unreachable.
0
0
0
The URL 'https://www.chateau-picquigny.com' is unreachable. Status code: 429
0
0
0
The URL 'https://www.chateaupirou.org' is unreachable.
0
0
0
The URL 'https://www.chateau-olhain.com' is unreachable.
The URL 'https://www.chateaumalbrouck.com' is unreachable.
0
The URL 'https://www.chateaugourdon.com' is unreachable.
The URL 'https://www.murolchateau.com' is unreachable.
0
The URL 'https://www.chateaufougeres.com' is unreachable.
The URL 'https://www.chateaubazoches.com' is unreachable.
The URL 'https://www.chateau-de-la-rochepot.com' is unreachable.
0
The URL 'https://www.chateaudedigoin.efr' is unreachable.
The URL 'https://www.chateauanger.sfr' is unreachable.
The URL 'https://www.suscini.ofr' is unreachable.
The URL 'https://www.chateauvillandr.yfr' is unreachable.
The URL 'https://www.chateaub